<a href="https://colab.research.google.com/github/giseletoledo/leitorTexto/blob/main/leitorPdf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
pip install PyPDF2


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.6 MB/s eta 0:00:00


In [2]:
pip install googletrans==4.0.0-rc1

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 35.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 3.9 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17397 sha256=5513f9842dda3ce4049acb71911ab69f1fff44524d52a688eb6377a9077db96d
  Stored in directory: /root/.cache/pip/wheels/c0/59/9f/7372f0cf70160fe61b528532e1a7c8498c4becd6bcffb022de
Successfully built googletrans
  Attempting uninstall: chardet
    Found existing installation: chardet 5.2.0
    Uninstalling chardet-5.2.0:


In [6]:
pip install TTS


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 45.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 5.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 41.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 54.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 45.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 73.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of librosa to determine which version is compatible with other re

In [1]:
import io
import logging
import re
import time
import os
from PyPDF2 import PdfReader
from googletrans import Translator
from TTS.api import TTS
from ipywidgets import FileUpload, Button, VBox
from IPython.display import display, HTML, Audio, clear_output

# Configuração do logging
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')

# Inicializando o modelo de TTS em espanhol
tts = TTS(model_name="tts_models/es/css10/vits")

def translate_with_retry(text, translator, dest='pt', max_retries=3, delay=2):
    """Realiza a tradução de um texto com múltiplas tentativas em caso de falha."""
    if text is None:
        logging.warning("Texto fornecido para tradução é None.")
        return ""

    for i in range(max_retries):
        try:
            text = text.strip()  # Remover espaços em branco no início e no fim
            if text:
                translation = translator.translate(text, dest=dest)
                if translation and translation.text:
                    return translation.text
                else:
                    logging.warning(f"Tradução falhou para a parte: {text}")
            else:
                logging.warning("Texto vazio fornecido para tradução.")
        except Exception as e:
            logging.error(f"Erro ao traduzir: {str(e)}")
        time.sleep(delay)
    return text

def extract_text_from_pdf(pdf_bytes):
    """Extrai texto de um PDF, ignorando imagens, por página."""
    try:
        reader = PdfReader(io.BytesIO(pdf_bytes))
        return reader.pages
    except Exception as e:
        logging.error(f"Erro ao extrair texto do PDF: {e}")
        return None

def format_text(text, page_number):
    """Formata o texto extraído para identificar títulos e subtítulos."""
    title_pattern = re.compile(r'^\d+\.\s.+', re.MULTILINE)
    subtitle_pattern = re.compile(r'^\d+\.\d+\s.+', re.MULTILINE)

    formatted_text = re.sub(title_pattern, r'<h1 style="color: #800020;">\g<0></h1>', text)
    formatted_text = re.sub(subtitle_pattern, r'<h2 style="color: darkblue;">\g<0></h2>', formatted_text)

    return formatted_text

def text_to_speech_tts(text, lang='es'):
    """Converte texto para fala usando TTS e retorna o arquivo de áudio."""
    try:
        # Gerar o áudio e salvar em um arquivo temporário
        audio_file_path = "output_original.wav"
        tts.tts_to_file(text=text, file_path=audio_file_path)

        return audio_file_path
    except Exception as e:
        logging.error(f"Erro ao gerar áudio com TTS: {e}")
        return None

def generate_audio(button, text):
    """Gera o áudio para o texto associado ao botão clicado e exibe o áudio."""
    try:
        audio_file_path = text_to_speech_tts(text)
        if audio_file_path:
            display(Audio(audio_file_path, autoplay=False))
            os.remove(audio_file_path)  # Limpar arquivo de áudio após exibir
    except Exception as e:
        logging.error(f"Erro ao gerar áudio: {e}")

def handle_pdf_upload(change):
    """Processa o upload do PDF, traduz o conteúdo e cria botões para gerar áudios."""
    try:
        if not change.new:
            display(HTML("Erro: Nenhum arquivo foi carregado."))
            return

        uploaded_filename = next(iter(change.new))
        content = change.new[uploaded_filename].get("content")

        if content is None:
            display(HTML("Erro: O conteúdo do arquivo está vazio ou não foi carregado corretamente."))
            return

        pages = extract_text_from_pdf(content)

        if not pages:
            display(HTML("Erro: Não foi encontrado texto no PDF ou houve um problema na extração."))
            return

        translator = Translator()

        # Limpar a saída anterior
        clear_output(wait=True)
        display(upload_btn)

        # Processar página por página
        for page_number, page in enumerate(pages, start=1):
            page_text = page.extract_text()
            if page_text:
                formatted_text = format_text(page_text, page_number)

                # Dividir o texto em partes menores para tradução
                text_parts = [formatted_text[i:i + 5000] for i in range(0, len(formatted_text), 5000)]
                translated_page = ""

                # Traduzir o texto em partes
                for i, part in enumerate(text_parts):
                    if part is not None:
                        try:
                            translated_part = translate_with_retry(part, translator)
                            translated_page += translated_part
                        except Exception as e:
                            logging.error(f"Erro ao traduzir parte {i+1} da página {page_number}: {e}")
                            translated_page += f"<p style='color: red;'>Erro na tradução da parte {i+1}: {part}</p>"
                    else:
                        translated_page += "<p style='color: red;'>Parte vazia ou não válida</p>"

                # Exibir texto original, tradução e botão para gerar áudio
                text_original = f"<h3>Texto Original</h3><p>{formatted_text}</p>"
                text_translated = f"<h3>Texto Traduzido</h3><p>{translated_page}</p>"
                button = Button(description=f'Gerar Áudio da Página {page_number}')
                button.on_click(lambda b, t=formatted_text: generate_audio(b, t))

                display(HTML(f"""
                <h2 style="color: #800020;">Página {page_number}</h2>
                <div style="background-color: #f0f0f0; padding: 10px; margin-bottom: 20px;">
                    {text_original}
                    {text_translated}
                </div><br>
                """))
                display(button)

    except Exception as e:
        logging.error(f"Erro ao processar o arquivo: {e}")
        display(HTML(f"Erro ao processar o arquivo: {str(e)}"))

# Criar o widget de upload de arquivo
upload_btn = FileUpload(accept='.pdf', multiple=False)
upload_btn.observe(handle_pdf_upload, names="value")

# Mostrar o botão de upload
display(upload_btn)

RuntimeError: module was compiled against NumPy C-API version 0x10 (NumPy 1.23) but the running NumPy has C-API version 0xf. Check the section C-API incompatibility at the Troubleshooting ImportError section at https://numpy.org/devdocs/user/troubleshooting-importerror.html#c-api-incompatibility for indications on how to solve this problem.

 > Downloading model to /root/.local/share/tts/tts_models--es--css10--vits


 97%|█████████▋| 97.9M/101M [00:02<00:00, 38.3MiB/s]

 > Model's license - bsd-3-clause
 > Check https://opensource.org/licenses for more info.
 > Using model: vits
 > Setting up Audio Processor...
 | > sample_rate:22050
 | > resample:False
 | > num_mels:80
 | > log_func:np.log10
 | > min_level_db:0
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:None
 | > fft_size:1024
 | > power:None
 | > preemphasis:0.0
 | > griffin_lim_iters:None
 | > signal_norm:None
 | > symmetric_norm:None
 | > mel_fmin:0
 | > mel_fmax:None
 | > pitch_fmin:None
 | > pitch_fmax:None
 | > spec_gain:20.0
 | > stft_pad_mode:reflect
 | > max_norm:1.0
 | > clip_norm:True
 | > do_trim_silence:False
 | > trim_db:60
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:10
 | > hop_length:256
 | > win_length:1024
 > initialization of speaker-embedding layers.
 > initialization of language-embedding layers.


/usr/local/lib/python3.10/dist-packages/TTS/utils/io.py:54: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(f, map_location=map_location, **kwargs)


FileUpload(value={}, accept='.pdf', description='Upload')